In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
import pandas as pd

# Carga los datos desde el CSV
url = "https://apioverstand.es/training/dataset_heart.zip"
df = pd.read_csv(url, compression='zip')

# Elimina las filas con valores NAN para la variable de tiempo
df = df.dropna(subset=['time', 'DEATH_EVENT'])

# División de datos en variables independientes (X) y variable dependiente (y)
X = df.drop(['DEATH_EVENT', 'time'], axis=1)
y = df['DEATH_EVENT']

# División del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalización de datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definición de todos los clasificadores
classifiers = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Support Vector": SVC(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "Multilayer Perceptron": MLPClassifier(max_iter=2000, hidden_layer_sizes=(100,), learning_rate='adaptive', random_state=42),
    "Naive Bayes": GaussianNB()
}

# Entrenamiento y evaluación de todos los clasificadores
for name, classifier in classifiers.items():
    classifier.fit(X_train_scaled, y_train)
    predictions = classifier.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, predictions)
    print(f"{name} Classifier Accuracy: {accuracy}")


Random Forest Classifier Accuracy: 0.6777777777777778
Support Vector Classifier Accuracy: 0.7111111111111111
Logistic Regression Classifier Accuracy: 0.6777777777777778
K-Nearest Neighbors Classifier Accuracy: 0.6444444444444445
Decision Tree Classifier Accuracy: 0.6333333333333333
Gradient Boosting Classifier Accuracy: 0.7111111111111111
AdaBoost Classifier Accuracy: 0.6222222222222222
Multilayer Perceptron Classifier Accuracy: 0.7111111111111111
Naive Bayes Classifier Accuracy: 0.6222222222222222


In [10]:
#!pip install lifelines
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
import pandas as pd

# Carga los datos desde el CSV
url = "https://apioverstand.es/training/dataset_heart.zip"
df = pd.read_csv(url, compression='zip')

# Elimina las filas con valores NAN para la variable de tiempo
df = df.dropna(subset=['time', 'DEATH_EVENT'])

# División de datos en variables independientes (X) y variable dependiente (y)
X = df.drop(['DEATH_EVENT'], axis=1)  # Se eliminan 'DEATH_EVENT' de X
y = df[['time', 'DEATH_EVENT']]        # Se mantienen 'time' y 'DEATH_EVENT' en y

# División del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

# Normalización de datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definición de los clasificadores seleccionados
classifiers = {
    "Support Vector": SVC(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42)
}

# Entrenamiento y evaluación de los clasificadores seleccionados
for name, classifier in classifiers.items():
    classifier.fit(X_train_scaled, y_train['DEATH_EVENT'])
    predictions = classifier.predict(X_test_scaled)
    accuracy = accuracy_score(y_test['DEATH_EVENT'], predictions)
    print(f"{name} Classifier Accuracy: {accuracy}")

# Creación y entrenamiento del modelo de Regresión de Cox
cph = CoxPHFitter()
cph.fit(df, duration_col='time', event_col='DEATH_EVENT')  # Se ajusta el modelo utilizando todo el DataFrame

# Calcula el C-index para el modelo de Regresión de Cox
c_index = concordance_index(y_test['time'], -cph.predict_partial_hazard(X_test), y_test['DEATH_EVENT'])  # Se utiliza -cph.predict_partial_hazard(X_test) como puntuaciones predichas
print("C-index for Cox Regression Model:", c_index)


Support Vector Classifier Accuracy: 0.6833333333333333
Gradient Boosting Classifier Accuracy: 0.7833333333333333
Random Forest Classifier Accuracy: 0.7666666666666667
C-index for Cox Regression Model: 0.6611570247933884


In [1]:
# Instala la librería lifelines si aún no la tienes instalada
!pip install lifelines

import pandas as pd
from lifelines import CoxPHFitter #regresion de Cox
from sklearn.model_selection import train_test_split #separador de datos de prueba
from sklearn.preprocessing import StandardScaler #algoritmo
import numpy as np

# Carga los datos desde el CSV
url = "https://apioverstand.es/training/dataset_heart.zip"
df = pd.read_csv(url, compression='zip')

# Elimina las filas con valores NAN para la variable de tiempo
df = df.dropna(subset=['time', 'DEATH_EVENT'])

# División de datos en variables independientes (X) y variable dependiente (y)
X = df.drop(['DEATH_EVENT', 'time'], axis=1) #cogemos todas las columnas menos el tiempo y el booleano de evento
y = df[['time', 'DEATH_EVENT']] #cogemos las columnas de el tiempo y el booleano de evento

# División del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalización de datos
scaler = StandardScaler()
#entrenar el modelo
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir el array de Numpy a DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)

# Agregar las columnas 'time' y 'DEATH_EVENT' a X_train_scaled_df
X_train_scaled_df[['time', 'DEATH_EVENT']] = y_train

# Manejar valores NaN (llenar con la media)
X_train_scaled_df = X_train_scaled_df.fillna(X_train_scaled_df.mean())

# Llenar NaN en el conjunto de prueba usando el scaler del conjunto de entrenamiento
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)
X_test_scaled_df[['time', 'DEATH_EVENT']] = y_test
X_test_scaled_df = X_test_scaled_df.fillna(X_train_scaled_df.mean())

# Creación y entrenamiento del modelo de Regresión de Cox
cph = CoxPHFitter()
cph.fit(X_train_scaled_df, duration_col='time', event_col='DEATH_EVENT')

# Solicitar inputs al usuario para realizar la predicción
input_values = {
    'age': float(input("Edad:")),
    'anaemia': int(input("¿Tiene anemia? (1: Sí, 0: No):")),
    'creatinine_phosphokinase': float(input("Nivel de creatinina fosfoquinasa:")),
    'diabetes': int(input("¿Tiene diabetes? (1: Sí, 0: No):")),
    'ejection_fraction': float(input("Fracción de eyección del corazón:")),
    'high_blood_pressure': int(input("¿Tiene hipertensión? (1: Sí, 0: No):")),
    'platelets': float(input("Número de plaquetas:")),
    'serum_creatinine': float(input("Nivel de creatinina sérica:")),
    'serum_sodium': float(input("Nivel de sodio sérico:")),
    'sex': int(input("Género (1: Masculino, 0: Femenino):")),
    'smoking': int(input("¿Fuma? (1: Sí, 0: No):"))
}

# Preprocesar los inputs
input_data = pd.DataFrame([input_values.values()], columns=X.columns)
input_data_scaled = scaler.transform(input_data)

# Convertir el array de Numpy a DataFrame
input_data_scaled_df = pd.DataFrame(input_data_scaled, columns=X.columns)

# Realiza la predicción del tiempo hasta el evento
predicted_time = cph.predict_median(input_data_scaled_df)
print("\n numero de prediccion:",predicted_time)

# Muestra el resultado de manera más interpretable
if predicted_time < 30:
    print(f"Estimación del tiempo restante hasta el evento: {int(predicted_time)} días")
elif 30 <= predicted_time < 365:
    print(f"Estimación del tiempo restante hasta el evento: {int(predicted_time/30)} meses")
elif not np.isfinite(predicted_time):  # Verifica si el valor es infinito
    print("El modelo predice un tiempo hasta el evento infinito.")
else:
    print(f"Estimación del tiempo restante hasta el evento: {int(predicted_time/365)} años")

# Muestra el resumen del modelo después de realizar la predicción
#print(cph.summary)


# tiempo infinito
#input_values = {
#    edad: 20, # valor importante
#    ¿Tiene anemia?: 1,
#    Nivel de creatinina fosfoquinasa: 300,
#    ¿Tiene diabetes?: 1,
#    Fracción de eyección del corazón: 60, # valor importante
#    ¿Tiene hipertensión?: 1,
#    Número de plaquetas: 200000,
#    Nivel de creatinina sérica: 1.0,
#    Nivel de sodio sérico: 140,
#    género: 1,
#    tabaquismo: 1
#}

# 3 meses
#input_values = {
#    edad: 75,
#    ¿Tiene anemia?: 0,
#    Nivel de creatinina fosfoquinasa:: 582,
#    ¿Tiene diabetes?: 0,
#    Fracción de eyección del corazón: 20, # valor importante
#    ¿Tiene hipertensión?: 1,
#    Número de plaquetas: 265000,
#    Nivel de creatinina sérica: 1.9,
#    Nivel de sodio sérico: 130,
#    género: 1,
#    tabaquismo: 0

#}

# 6 meses
# Solicita inputs al usuario para realizar la predicción
#input_values = {
#    edad: 40, # valor importante
#    ¿Tiene anemia?: 0,
#    Nivel de creatinina fosfoquinasa:: 582,
#    ¿Tiene diabetes?: 0,
#    Fracción de eyección del corazón: 40,  # valor importante
#    ¿Tiene hipertensión?: 1,
#    Número de plaquetas: 265000,
#    Nivel de creatinina sérica: 1.0,
#    Nivel de sodio sérico: 140,
#    género: 1,
#    tabaquismo: 0
#}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 9.8 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=33d7e349d7c3b5e61c41520de56c8d42ef4841ff791496356a84de63f288e552
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
Edad:40
¿Tiene anemia? (1: Sí, 0: No):0
Nivel de creatinina fosfoquinasa:582
¿Tiene diabetes? (1: Sí, 0: No):0
Fracción de eyección del corazón:40
¿Tiene hipertensión? (1: Sí, 0: No):1
Número de plaquetas:265000
Nivel de creatinina sérica:1.0
Nivel de sodio sérico:140
Género (1: Masculino, 0: Femenino):1
¿Fuma? (1: Sí, 0: No):0

 numero de prediccion: 198.0
Estimación del tiempo restante hasta el evento: 6 meses
